In [25]:
### Naveen Rajan P
### MTech Data Science, Amity University
### BTech + MTech Dual degree Mechanical, IIITDMK
### Extreme Learning Machine code for brain tumor detection

In [26]:
# Packages used
import os
import glob
import shutil
import random
import pandas as pd
from PIL import Image
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
import time

In [27]:
# Parameters to modify
ims1 = 130
tts = 0.11
hu = 1000
rel = 1

In [28]:
# Importing images
def importing_data(path):
    sample = []
    for filename in tqdm(glob.glob(path)):
        img = Image.open(filename,'r')
        img = img.resize((ims1,ims1))
        img = np.array(img)
        IMG = img.ravel()
        sample.append(IMG)
    return sample
t = time.time()
path1 = '//kaggle/input/brain-tumor-detection/no/*.jpg'
path2 = '//kaggle/input/brain-tumor-detection/yes/*.jpg'
path3 = '//kaggle/input/brain-tumor-detection/pred/*.jpg'

print("importing training no")
train_n = importing_data(path1)
print("importing training yes")
train_y = importing_data(path2)


  2%|▏         | 31/1500 [00:00<00:04, 303.18it/s]

importing training no


  2%|▏         | 24/1500 [00:00<00:06, 238.94it/s]

importing training yes


100%|██████████| 1500/1500 [00:07<00:00, 195.57it/s]


In [29]:
# Importing more images into the dataset
path11 = '//kaggle/input/brain-tumor-classification-mri/Testing/glioma_tumor/*.jpg'
path21 = '//kaggle/input/brain-tumor-classification-mri/Testing/meningioma_tumor/*.jpg'
path31 = '//kaggle/input/brain-tumor-classification-mri/Testing/pituitary_tumor/*.jpg'
path41 = '//kaggle/input/brain-tumor-classification-mri/Testing/no_tumor/*.jpg'

# print("importing training no")
# x = (importing_data(path41))
# for i in x:
#     train_n.append(i)
print("importing training yes")
x = (importing_data(path11))
for i in x:
    train_y.append(i)
print("importing training yes")
x = (importing_data(path21))
for i in x:
    train_y.append(i)
print("importing training yes")
x = (importing_data(path31))
for i in x:
    train_y.append(i)

  9%|▉         | 9/100 [00:00<00:01, 87.20it/s]

importing training yes


 10%|▉         | 11/115 [00:00<00:00, 108.52it/s]

importing training yes


  8%|▊         | 6/74 [00:00<00:01, 58.36it/s]

importing training yes


100%|██████████| 74/74 [00:01<00:00, 57.64it/s]


In [30]:
# Importing more images into the dataset
path11 = '//kaggle/input/brain-tumor-classification-mri/Training/glioma_tumor/*.jpg'
path21 = '//kaggle/input/brain-tumor-classification-mri/Training/meningioma_tumor/*.jpg'
path31 = '//kaggle/input/brain-tumor-classification-mri/Training/pituitary_tumor/*.jpg'
path41 = '//kaggle/input/brain-tumor-classification-mri/Training/no_tumor/*.jpg'

# print("importing training no")
# x = (importing_data(path41))
# for i in x:
#     train_n.append(i)
print("importing training yes")
x = (importing_data(path11))
for i in x:
    train_y.append(i)
print("importing training yes")
x = (importing_data(path21))
for i in x:
    train_y.append(i)
print("importing training yes")
x = (importing_data(path31))
for i in x:
    train_y.append(i)

  1%|          | 9/826 [00:00<00:10, 78.89it/s]

importing training yes


  1%|          | 10/822 [00:00<00:09, 89.50it/s]

importing training yes


  1%|          | 9/827 [00:00<00:10, 79.71it/s]

importing training yes


100%|██████████| 827/827 [00:10<00:00, 81.12it/s]


In [31]:
print("time taken to load all images:",time.time()-t)

time taken to load all images: 47.10554051399231


In [32]:
df_train_n = pd.DataFrame({'image':train_n, 'label': 'Healthy'})
df_train_y = pd.DataFrame({'image':train_y, 'label': 'Affected'})

In [33]:
# Combining the data
train_data = df_train_n.append(df_train_y,ignore_index=True)

In [34]:
# Ignoring images that are very oddly shaped to avoid outliers
c = 1
igs = []
for i in range(len(train_data)):
    if(len(train_data.image[i])!=(3*ims1*ims1)):
#         print(c,i)
        c = c + 1
        igs.append(i)
# print(igs)
print(len(igs))

109


In [35]:
train_data=train_data.drop(igs,axis=0)

In [36]:
x_train = []
for tdim in train_data.image:
    x_train.append(tdim)
y_train = np.array(train_data.label)

In [37]:
y_traind = []
for i in range(len(y_train)):
    if(y_train[i]=="Healthy"):
        y_traind.append(0)
    else:
        y_traind.append(1)
print(len(y_traind))

5655


In [38]:
x_traind = np.array(x_train)

In [39]:
# One hot encoding
CLASSES = 2
y_train = np.zeros([len(y_traind), CLASSES])
for i in range(len(y_traind)):
        y_train[i][y_traind[i]] = 1

In [40]:
# Test train splitting
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x_traind, y_train, test_size=tts)
print('Train size: {train}, Test size: {test}'.format(train=len(x_train), test=len(x_test)))

Train size: 5032, Test size: 623


In [41]:
# Changing to Matrix format
x_train=np.matrix(x_train)
x_test=np.matrix(x_test)
y_train=np.matrix(y_train)
y_test=np.matrix(y_test)

In [42]:
# Starting to build ELM model
t = time.time()
INPUT_LENGTH = x_train.shape[1]
HIDDEN_UNITS = hu #hidden units

Win = np.random.normal(size=[INPUT_LENGTH, HIDDEN_UNITS])
Win = np.matrix(Win)

In [43]:
def input_to_hidden(x):
    a = np.dot(x, Win)
    a = np.maximum(a, rel, a) # ReLU activation
    return a

In [44]:
X = input_to_hidden(x_train)
Xt = np.transpose(X)

In [45]:
# Moore-Penrose pseudo inverse

www1 = np.dot(Xt, X)

b = np.identity(len(np.array(www1)))
b = np.dot(b,3)

www1 = www1 + b
ww1 = np.linalg.inv(www1)
ww2 = np.dot(Xt, y_train)

Wout = np.dot(ww1, ww2)

In [46]:
def predict(x):
    x = input_to_hidden(x)
    y = np.dot(x, Wout)
    return y

In [47]:
print("time taken to build model:",time.time()-t)

time taken to build model: 9.975350141525269


In [48]:
# Prediction
t = time.time()
y = predict(x_test)
print("time taken for prediction:",time.time()-t)
correct = 0
total = y.shape[0]
for i in range(total):
    predicted = np.argmax(y[i])
    test = np.argmax(y_test[i])
    correct = correct + (1 if predicted == test else 0)
print('Accuracy: {:f}'.format(correct*100/total))

time taken for prediction: 0.993018388748169
Accuracy: 96.147673
